# Classification: Distances

Exploring euclidean distances between facial points caputred by nuiCapture as candidate features.
<p>
    <img src="nuicapture.png">
    <em>Source: CadavidConcepts</em>
</p>

In [1]:
import glob as gl
import numpy as np
from scipy.io import loadmat
from typing import List
import pandas as pd
from scipy.spatial import distance
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
def labelname(file_name):
    label = file_name.replace("data/points/sample", "").lower()
    return label.replace(".mat", "")


class Signal:
    def __init__(self, x, y, label):
        self.x = x
        self.y = y
        self.label = label

## Dataset preparation

In [3]:
files = gl.glob("data/points/*.mat")  # type: list
signals = []  # type: List[Signal]

for f in files:
    data = loadmat(f).get('pontosSinal')
    signals.append(Signal(data[:, ::2], data[:, 1::2], labelname(f)))

nsigns = len(signals)
nrecs, nx = np.shape(signals[0].x)  # Number of recordings and number of features
nframes = 5

signals_feat = []  # Updated signals, according to each experiment
signals_labels = []
labels_dict = {}  # Dictionary of signals' labels, for reference

points = [3,  6,  8,  9, 11, 16, 17, 20, 25, 32, 49, 50, 53, 58, 65, 91, 92]
points = np.array(points)
npoints = np.shape(points)[0]

frame_x = []
frame_y = []

i = 0
for s in signals:
    frame_x.append(np.split(s.x, 5, axis=1))
    frame_y.append(np.split(s.y, 5, axis=1))
    signals_labels.append([i] * nrecs)
    labels_dict[i] = s.label
    i += 1

frame_x = np.array(frame_x)
frame_y = np.array(frame_y)
new_frame_x = []
new_frame_y = []

for s in range(nsigns):
    new_frame_x.append(np.transpose(frame_x[s,:,:,points]))
    new_frame_y.append(np.transpose(frame_y[s,:,:,points]))

sig_features = np.empty((nrecs*nsigns, nframes*npoints*2))
sig_features[:,::2] = (np.vstack(new_frame_x)).reshape(nrecs*nsigns,npoints*5)
sig_features[:,1::2] = (np.vstack(new_frame_y)).reshape(nrecs*nsigns,npoints*5)

sig_labels = np.reshape(signals_labels, (nsigns * nrecs,))

np.savetxt("data/csv/distances-labels.csv",sig_labels,delimiter=",")
np.savetxt("data/csv/distances.csv",sig_features,delimiter=",")

## Classification

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from pprint import pprint

In [5]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start=800, stop=2000, num=10)]
# Number of features to consider at every split
max_features = ['sqrt', 0.3, 0.2]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 80, num=10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [3, 5, 7]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 3, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Criterion
criterion = ['gini', 'entropy']
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion': criterion
              }

In [ ]:
niter = 30
results = []
train_acc = []
test_acc = []
train_report =[]
class_report = []
selected_params = []
cm = [] # confusion matrix
feature_importance = []
col_names = range(1,1211)

for i in range(niter):
    print("Iteration:: ", i)
    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=42)
    sss.get_n_splits(sig_features, sig_labels)

    for train_index, test_index in sss.split(sig_features, sig_labels):
        train_x, test_x = sig_features[train_index], sig_features[test_index]
        train_y, test_y = sig_labels[train_index], sig_labels[test_index]

    rf = RandomForestClassifier()
    rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=500, cv=3, verbose=1, n_jobs=-1)

    rf_random.fit(train_x, train_y)
    predictions = rf_random.predict(test_x)
    
    selected_params.append(rf_random.best_params_)
    train_acc.append(accuracy_score(train_y, rf_random.predict(train_x)))
    test_acc.append(accuracy_score(test_y, predictions))
    cm.append(pd.crosstab(test_y, predictions, rownames=['True'], colnames=['Predicted'], margins=True))
    class_report.append(classification_report(test_y, predictions, target_names=list(labels_dict.values())))
    feature_importance.append(pd.DataFrame(data=sorted(zip(map(lambda x: round(x, 4), 
                                                               rf_random.best_estimator_.feature_importances_), 
                                                           col_names), reverse=True)))

Iteration::  0
Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 25.3min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 40.6min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed: 49.4min finished
/anaconda/envs/tensorflow/lib/python3.4/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Iteration::  1
Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 16.7min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 29.8min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 46.3min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed: 55.5min finished


Iteration::  2
Fitting 3 folds for each of 500 candidates, totalling 1500 fits


...

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   56.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 22.4min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 34.7min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed: 40.8min finished


Iteration::  24
Fitting 3 folds for each of 500 candidates, totalling 1500 fits


/anaconda/envs/tensorflow/lib/python3.4/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 21.3min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 34.6min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed: 41.8min finished
/anaconda/envs/tensorflow/lib/python3.4/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Iteration::  25
Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.1min


## Results

In [ ]:
from functools import reduce

conf_matrix = reduce(lambda x, y: x.add(y, fill_value=0), cm)

np.savetxt("results/distances/dst-trainacc.csv",train_acc,delimiter=",")
np.savetxt("results/distances/dst-testacc.csv",test_acc,delimiter=",")
np.savetxt("results/distances/dst-selected_params.txt",selected_params,fmt='%s',delimiter='\n')
np.savetxt("results/distances/dst-classreport.txt",class_report,fmt='%s',delimiter='\n')
conf_matrix.to_csv("results/distances/dst-cm.csv")

import pickle
with open('results/distances/dst-featimportance.pickle', 'wb') as fp:
    for feat in feature_importance:
        pickle.dump(feat, fp)

In [ ]:
acc = np.transpose([train_acc, test_acc])
acc = pd.DataFrame(data=acc, columns=['train_acc', 'test_acc'])
boxplot = acc.boxplot(column=['train_acc', 'test_acc'])
plt.show()

In [ ]:
np.array(np.multiply(acc['test_acc'],100))

import matplotlib.pyplot as plt
%matplotlib notebook
acc_traj = pd.DataFrame(data=np.transpose([np.array(np.multiply(acc['test_acc'],100)), 
                                           [65,70,65,60,75,65,65,70,50,40,55,75,80,75,65,65,90,70,65,50,65,55,75,70,70,45,75,75,60,70], 
                                           [45,45,45,50,55,55,60,40,45,45,40,60,45,45,35,50,60,65,55,55,40,55,55,50,60,55,60,55,50,50]]), 
                                           columns=['RF', 'SVM', 'k-NN'])
boxplot_traj = acc_traj.boxplot(column=['RF', 'SVM', 'k-NN'], vert=False)
plt.show()

In [ ]:
acc.describe()